In [1]:
import pymysql
import pandas as pd
import numpy as np

In [ ]:
conn=pymysql.connect(host='mysql-backup.shanzhen.me',
                     user='readonly_medicine_liuqq',
                     password="r3g1^QWgV8GeYJw4",
                     database='asgard_dataplatform',
                     port= 3001,
                     charset='utf8')

In [ ]:
dict = pd.read_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\dict2_conserv.xlsx")
id_list = dict.loc[:,'id'].values.tolist()

In [ ]:
tag_df = pd.DataFrame(columns=['ORDER_CODE', 'TAG_ID'])

In [ ]:
for items in id_list:
    id_str = "'"+ items + "'"
    tagsql = 'SELECT ORDER_CODE,TAG_ID \
    FROM T66_APOLLO_ANALYSIS_TAG \
    WHERE SOURCE_TYPE = 2 \
    AND SZ_STATUS = 1 AND TAG_ID IN ('+id_str+') '
    tag = pd.read_sql(tagsql,con=conn)
    tag_df = pd.concat([tag_df,tag],axis = 0) 
    
tag_df.to_pickle(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\4.pkl")

In [4]:
renaltag = pd.read_pickle(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\tagall.pkl")

In [30]:
renaltag

,ORDER_CODE,TAG_ID,value
0,068138154589,无回声,1
1,348099327682,无回声,1
2,103794055941,无回声,1
3,782720070666,无回声,1
4,570550235175,无回声,1
...,...,...,...
196,8kg994h55usaiam,值,1
197,8kg9evxglmnhkee,值,1
198,8kganyvzeonrvxn,值,1
199,8kgaxcfpxcc69bk,值,1


In [96]:
renal = pd.read_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201016\ranal_dict.xlsx")

In [95]:
dict1 = renal.set_index('tag_id').to_dict()['abbr']
dict2 = renal.set_index('tag_id').to_dict()["loc"]

In [29]:
renaltag['value']=1
renaltag.replace(dict,inplace=True)

C:\Users\13670\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\13670\Anaconda3\lib\site-packages\pandas\core\frame.py:4384: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [97]:
desclist = renal.loc[renal['logic']== "A",'abbr'].tolist()
summlist = renal.loc[renal['logic']=="a",'abbr'].tolist()

In [102]:
dfsumm.TAG_ID.drop_duplicates()

0     结晶
0     结石
0     囊肿
0    错构瘤
0    钙乳症
0     占位
0    钙化灶
0     结节
Name: TAG_ID, dtype: object

In [103]:
renaltag = renaltag.drop_duplicates(subset = ["ORDER_CODE","TAG_ID"])

In [104]:
dfdesc = renaltag.loc[renaltag['TAG_ID'].isin(desclist)]
dfsumm = renaltag.loc[renaltag['TAG_ID'].isin(summlist)]

In [105]:
ordercode=[]
desclist=[]
summlist=[]
for i in dfsumm['ORDER_CODE'].drop_duplicates():
    desclistsub=dfdesc.loc[dfdesc['ORDER_CODE']==i,'TAG_ID'].tolist()
    summlistsub=dfsumm.loc[dfsumm['ORDER_CODE']==i,'TAG_ID'].tolist()
    ordercode.append(i)
    summlist.append(summlistsub)
    desclist.append(desclistsub)
    
renaltotal = pd.DataFrame({'ordercode':ordercode,'summlist':summlist,'desclist':desclist}) 

In [108]:
renaltotal.to_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201016\renaltotal.xlsx")

In [ ]:
df = renaltotal.applymap(lambda x: tuple(x))

In [106]:
renaltotal_tuple = pd.DataFrame(df.groupby(['summlist','desclist'])['ordercode'].count())

In [107]:
renaltotal_tuple.to_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201016\renal_tuple_.xlsx")

In [61]:
solid = pd.read_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201016\solid.xlsx")

In [62]:
solid.head()

,summ,desc,solid
0,钙乳症,高回声,1
1,钙乳症,囊性回声,0
2,钙乳症,混合回声,0
3,钙乳症,无回声,0
4,钙乳症,等回声,0


In [45]:
renaltotal_tuple = pd.read_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201016\renal_tuple_.xlsx")

In [46]:
renaltotal_tuple.head()

,summlist,desclist,ordercode
0,"('占位',)",(),112
1,"('占位',)","('低回声',)",19
2,"('占位',)","('低回声', '回声不均匀')",6
3,"('占位',)","('强回声',)",2
4,"('占位',)","('强回声', '低回声', '回声不均匀')",1


In [72]:
desc_list = []
summ_list = []
count_list = []
source_model = []

for line in renaltotal_tuple.values:
    desc = eval(line[0])
    summ = eval(line[1])
    count = line[2]    
    if len(summ)==0:
        source = "summary_missing"
        for index in range(len(desc)):
            descsub = desc[index]
            desc_list.append(descsub)
            summ_list.append(0)
            count_list.append(count)
            source_model.append(source)                
    elif (len(desc)== 1 & len(summ)==1):
        source = "complete"
        desc_list.append(desc[0])
        summ_list.append(summ[0])
        count_list.append(count)
        source_model.append(source)
    else :
        source = "split"
        for index_desc in range(len(desc)):
            descsub = desc[index_desc]
            for index_summ in range(len(summ)):
                summsub = summ[index_summ]
                countsub = count/(len(desc)*len(summ))
                desc_list.append(descsub)
                summ_list.append(summsub)
                count_list.append(countsub)
                source_model.append(source)
                
final = pd.DataFrame({"summ":desc_list,"desc":summ_list,"count":count_list,"source_model":source_model}) 
final

,summ,desc,count,source_model
0,占位,0,112.000000,summary_missing
1,占位,低回声,19.000000,complete
2,占位,低回声,3.000000,split
3,占位,回声不均匀,3.000000,split
4,占位,强回声,2.000000,complete
...,...,...,...,...
3669,钙化灶,高回声,0.111111,split
3670,钙化灶,回声不均匀,0.111111,split
3671,结节,强回声,0.111111,split
3672,结节,高回声,0.111111,split


In [78]:
final_ = pd.merge(final,solid,left_on =[ "summ","desc" ] ,right_on = ["summ","desc"])

In [80]:
final_.to_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201016\renal_final.xlsx")

In [87]:
class_dict = pd.read_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201016\ranal_dict.xlsx",sheet_name = "decision")

In [89]:
class_df = pd.merge(final_,class_dict,left_on = ["source_model","solid"],right_on = ["source_model","solid"])

In [92]:
class_groupby = pd.DataFrame(class_df.groupby(["summ","desc","source_model","solid","result"])["count"].sum())

In [93]:
class_groupby

count
summ desc   source_model    solid result            
占位   0      summary_missing 0     Error   167.000000
     低回声    complete        1     OK       19.000000
            split           1     Maybe     6.177778
     囊性回声   split           1     Maybe     0.483333
     回声不均匀  split           1     Maybe    16.538889
...                                              ...
错构瘤  等回声    complete        0     NO        5.000000
            split           0     NO        0.916667
     集合系统分离 split           0     NO        6.819444
     高回声    complete        2     OK      915.000000
            split           2     OK      157.427778

[149 rows x 1 columns]

In [94]:
class_groupby.to_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201016\ranal_groupby.xlsx")

In [123]:
high_risk = pd.read_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201016\mis with high_risk.xlsx")

In [121]:
renaltotal_or = pd.read_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201016\renaltotal.xlsx",index_col = 0)

In [138]:
renaltotal_or1 = renaltotal_or.apply(lambda x: tuple(eval(x)))

TypeError: eval() arg 1 must be a string, bytes or code object

In [124]:
order_code  = pd.merge(high_risk,renaltotal_or)

In [134]:
renaltotal_tuple = pd.DataFrame(renaltotal_or.groupby(['summlist','desclist'])['ordercode'].count())

In [137]:
renaltotal_tuple.to_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201016\renaltotal_list.xlsx")